## Fetch HIFLD Data
The Homeland Infrastructure Foundation-Level Data (HIFLD) has an Open Data platform providing access to numerous useful datasets. Here we explore how to download those datasets via the ArcGIS Python API.

If you navigate to the [HIFLD Open Data site](https://hifld-geoplatform.opendata.arcgis.com/) and search for "Electric Substations", you'll be taken to the Electric Substations Open Data access point: https://hifld-geoplatform.opendata.arcgis.com/datasets/electric-substations. Here, you'll see links to download the dataset in a choice of formats; you'll also see a link for the dataset's GeoService and GeoJSON APIs. The **GeoService API** is the ESRI REST endpoint for the data, a gateway for interacting or downloading the data. The **GeoJSON API** link will allow you to download the dataset directly in GeoJSON format. 

The code below outlines the steps for accessing the data via the GeoService API as it is more flexible, allowing us to query specific records if we want. First, however, we need to gather a bit of information from the GeoService API site:

### Navigating an GeoService API link
* Open the GeoService API [link](https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Electric_Substations_1/FeatureServer/0/query?outFields=*&where=1%3D1). 
 * *This site is an executed query of the data listing all the features. We want to back up a bit...*
* At the top of the page are the sequence of links leading up to this query. Navigate backwards to the [Substations](https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Electric_Substations_1/FeatureServer/0) link. 
 * This page shows information about the substations' **feature layer service**, revealing some useful information about the geoservice. 
 * Take note of the URL of this page. We can use this URL to access the data in Python using the ArcGIS Python API.

In [1]:
#Import the arcgis FeatureLayer class
from arcgis.features import FeatureLayer

/opt/conda/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
#Create a FeatureLayer object, using the feature layer service's URL
serviceURL = 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Electric_Substations_1/FeatureServer/0'
substations_Layer = FeatureLayer(theURL)

The [FeatureLayer object](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer) allows us to interact with the features. Our objective here is (1) to subset those substations falling within North Carolina, and then (2) save those records to a local shapefile. 

To subset the records, it helps to find out what fields are included in the feature layer, find what unique values are contained in that field, and then set up a query to extract the records we want in that dataset. Below are code examples showing how that's done. 

In [13]:
#Get a list fields 
allFields = substations_Layer.properties.fields
print(len(allFields))

24


In [14]:
#Now loop through each field in the "allFields" list and printing the field's name
for field in allFields:
    print(field.name)

OBJECTID
ID
NAME
CITY
STATE
ZIP
TYPE
STATUS
COUNTY
COUNTYFIPS
COUNTRY
LATITUDE
LONGITUDE
NAICS_CODE
NAICS_DESC
SOURCE
SOURCEDATE
VAL_METHOD
VAL_DATE
LINES
MAX_VOLT
MIN_VOLT
MAX_INFER
MIN_INFER


We see that one of the fields is `STATE`. Let's see how each state name is listed in this dataset.

In [16]:
#List the unique values of the STATE field
print(substations_Layer.get_unique_values('STATE'))

['TX', 'KS', 'PA', 'WI', 'DE', 'AB', 'IN', 'IL', 'NH', 'BC', 'MD', 'DC', 'CO', 'MI', 'VA', 'ME', 'AK', 'SC', 'RI', 'GU', 'WA', 'MA', 'MP', 'NV', 'NY', 'OH', 'VI', 'SD', 'CT', 'LA', 'NJ', 'MO', 'OR', 'CA', 'MN', 'MT', 'OK', 'PR', 'FL', 'AR', 'IA', 'VT', 'AL', 'ND', 'ID', 'GA', 'AZ', 'MS', 'TN', 'NC', 'NE', 'WV', 'WY', 'HI', 'SK', 'NM', 'KY', 'UT']


So, states are listed as their two digit abbreviation. Now we can create our query to subset only records that fall within North Carolina. 

In [17]:
#Subset North Carolina features
nc_Substations = substations_Layer.query("STATE = 'NC'")

In [19]:
#Reveal the type of feature returned
type(nc_Substations)

arcgis.features.feature.FeatureSet

The result of our query is a new type of object, a [FeatureSet](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset). The FeatureSet can easily be converted to a [Spatial Dataframe](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#spatialdataframe) (via the [sdf](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureSet.sdf) function, which, in turn, as a function [to_featureclass](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.SpatialDataFrame.to_featureclass) that exports the data as a shapefile. 

In [20]:
nc_Substations_sdf = nc_Substations.sdf

In [21]:
nc_Substations_sdf.spatial.to_featureclass(location=)

/opt/conda/lib/python3.7/site-packages/IPython/core/oinspect.py:601: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  info = self._info(obj, oname=oname, info=info, detail_level=detail_level)


In [1]:
import arcgis
print(arcgis.__version__)

1.6.2
